# Daum Cafe crawling

In [1]:
# import the libraries
import time
import re
import random as rd
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
import pickle
import datetime

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Driver setting
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
headers = {
    "User-Agent": user_agent
}

def setWebdriver():
    service = ChromeService(executable_path=ChromeDriverManager().install())    # 크롬 드라이버 최신 버전 설정

    options = ChromeOptions()
    options.add_argument('user-agent=' + user_agent)
    options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    # options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
    #options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
    #options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
    #options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    driver = webdriver.Chrome(service=service, options=options)

    return driver

In [3]:
# Scroll down
def scrollDown(driver, whileSeconds): 
    end = driver.execute_script("return document.body.scrollHeight")
    # while (True):
    #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #     time.sleep(rd.uniform(0.8, 1.2))
    #     new_end = driver.execute_script("return document.body.scrollHeight")
    #     if new_end == end:
    #         break
    #     end = new_end

    start = datetime.datetime.now() # 스크롤 다운 시작 시간 설정
    end = start + datetime.timedelta(seconds=whileSeconds) # 스크롤 다운 종료 시간 설정
    with tqdm(total=whileSeconds, desc='Scrolling Down...', leave='False') as pbar:
        while (datetime.datetime.now() < end):
            # 페이지 맨 아래로 스크롤 다운
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(1)        
            pbar.update(1)

    return

In [4]:
# make URL
def makeURL(query, start_date, end_date, page: int):
    # URL 설정
    url = f'https://search.daum.net/search?DA=STC&col=cafe&ed={end_date}235959&period=u&q={query}&sd={start_date}000000&w=fusion&p={page}'
    return url

In [5]:
# get title&URL
def getTitleURL(driver):
    global query, start_date, end_date, pages
    
    title_list = []
    url_list = []
    # 검색어, 시작 날짜, 종료 날짜 입력
    query = input("검색어를 입력하세요: ") # 검색어 입력, keyword: 우크라이나
    start_date = re.sub(r'[^0-9]', '', input("시작 날짜를 yyyy.mm.dd 형식으로 입력하세요: ")) # 시작 날짜(정규식을 이용하여 숫자만 추출)
    end_date = re.sub(r'[^0-9]', '', input("종료 날짜를 yyyy.mm.dd 형식으로 입력하세요: "))    # 종료 날짜(정규식을 이용하여 숫자만 추출)
    pages = int(input("크롤링할 기사 갯수를 입력하세요(15개 단위): ")) // 15 # 크롤링할 페이지 수 입력

    for i in tqdm(range(pages), desc="Colleting title and URL..."):
        url = makeURL(query, start_date, end_date, page=i+2)
        driver.get(url)
        time.sleep(rd.uniform(0.5, 1.0))
        # 블로그 글 element 수집
        try:
            articles = driver.find_elements(By.CLASS_NAME, "item-title");   #print(articles)
        except:
            print(f'articles error...')

        # title, url 수집
        for article in articles:    
            try:
                # title
                title = article.text;      title_list.append(title);       #print(f'Title: {title}')
            except:
                print(f'title error...')
            try:
                # url
                url = article.find_element(By.TAG_NAME, 'a').get_attribute("href");
                url_list.append(url);           #print(f'URL: {url}')
            except:
                print(f'URL error...')
        
        # print('-'*200)

    return title_list, url_list

In [6]:
# blog contents crawling
def getContents(driver, title_list, url_list):
    mainText_list = [] # 블로그 본문 리스트 초기화
    commentCnt_list = [] # 댓글 수 리스트 초기화
    comment_list = []   # 댓글 리스트 초기화
    imgCnt_list = []    # 이미지 수 리스트 초기화     
    img_list = []    # 이미지 리스트 초기화
    videoCnt_list = []  # 동영상 수 리스트 초기화
    video_list = [] # 동영상 리스트 초기화

    for i, url in enumerate(tqdm(url_list, desc="Contents...")):
        driver.get(url)
        time.sleep(1) # 로딩 대기

        # print(f'Title: {title_list[i]}')

        iframes = driver.find_elements(By.TAG_NAME, "iframe")    # iframe 수집
        if len(iframes) > 0:    # iframe이 있는지 확인
            # for iframe in iframes:
            #     print(f'iframe name: {iframe.get_attribute("name")}')  # iframe 이름 출력
            try:
                driver.switch_to.frame('down') # iframe 요소로 전환
            except:
                pass
            
        # 블로그 본문 수집
        try:
            main_text = driver.find_element(By.ID, "user_contents").text    # 블로그 본문 텍스트 수집, se-main-container, ContentRenderer
            mainText_list.append(main_text)    # 블로그 본문 리스트에 추가
            # print(f'Main text collected!');  # print(f'Main text: {main_text}'); 
        except NoSuchElementException as e:
            main_text = driver.find_element(By.ID, "bbs_contents").text    # 블로그 본문 텍스트 수집, se-main-container, ContentRenderer
            mainText_list.append(main_text)    # 블로그 본문 리스트에 추가
        except:
            mainText_list.append(None) # 블로그 본문이 없는 경우 None 추가
            
        # 댓글 수 수집
        try:
            comment_cnt = int(driver.find_element(By.CLASS_NAME, "num_tit").text)    # 댓글 수 수집
            commentCnt_list.append(comment_cnt) # 댓글 수 리스트에 추가
            # print(f'Comment count: {comment_cnt}')
        except NoSuchElementException as e:
            comment_cnt = int(driver.find_element(By.CLASS_NAME, 'txt_num').text)    # 댓글 수 수집
            commentCnt_list.append(comment_cnt) # 댓글 수 리스트에 추가
            # print(f'Comment count: {comment_cnt}')
        except:
            print(f'comment count error..., URL: {url}')

        # 댓글 수집
        if (comment_cnt > 0):
            try:
                comments = [comment.text for comment in driver.find_elements(By.CLASS_NAME, "desc_info")]    # 댓글 수집
                comment_list.append(comments)   # 댓글 리스트에 추가
                # print(f'comments({len(comments)}): {comments}')
            except:
                print(f'comments error2..., URL: {url}')
        else:
            comment_list.append(None)   # 댓글이 없는 경우 None 추가

        # 이미지 수집
        try:
            images = [img.get_attribute('src') for img in driver.find_element(By.ID, 'user_contents').find_elements(By.TAG_NAME, "img")]    # 이미지 수집
            imgCnt_list.append(len(images)) # 이미지 수 리스트에 추가
            if (len(images) > 0):
                img_list.append(images); 
                # print(f'Images({len(images)}): {images})')
            else:
                img_list.append(None);        
                # print(f'There is no images...')
        except NoSuchElementException as e:
            images = [img.get_attribute('src') for img in driver.find_element(By.ID, 'mArticle').find_elements(By.TAG_NAME, "img")]    # 이미지 수집
            imgCnt_list.append(len(images)) # 이미지 수 리스트에 추가
            if (len(images) > 0):
                img_list.append(images); 
                # print(f'Images({len(images)}): {images})')
            else:
                img_list.append(None);        
                # print(f'There is no images...')
        except:
                img_list.append(None)
                print(f'images error..., URL: {url}')

        # 동영상 수집
        try:
            temp = [video.get_attribute('src') for video in driver.find_element(By.ID, 'user_contents').find_elements(By.CSS_SELECTOR, "iframe[src]")]    # 동영상 수집
            videos = [video for video in temp if ('player' in video) or ('you' in video)]    # 동영상 URL에 'player' or 'youtu'가 있는지 확인
            videoCnt_list.append(len(videos))   # 동영상 수 리스트에 추가
            if (len(videos) > 0): 
                video_list.append(videos)
                # print(f'Videos({len(videos)}): {videos})')
            else: 
                video_list.append(None)
                # print(f'There is no videos...')
        except NoSuchElementException as e:
            temp = [video.get_attribute('src') for video in driver.find_element(By.ID, 'mArticle').find_elements(By.CSS_SELECTOR, "iframe[src]")]    # 동영상 수집
            videos = [video for video in temp if ('player' in video) or ('you' in video)]    # 동영상 URL에 'player' or 'youtu'가 있는지 확인
            videoCnt_list.append(len(videos))   # 동영상 수 리스트에 추가
            if (len(videos) > 0): 
                video_list.append(videos)
                # print(f'Videos({len(videos)}): {videos})')
            else: 
                video_list.append(None)
                # print(f'There is no videos...')
        except:
            print(f'videos error..., URL: {url}')
            
        

    # print('-'*200)
    time.sleep(rd.uniform(0.1, 0.3))

    return mainText_list, commentCnt_list, comment_list, imgCnt_list, img_list, videoCnt_list, video_list

In [7]:
if __name__ == '__main__':

    # webdriver init
    driver = setWebdriver()

    # title, url collecting
    title_list, url_list = getTitleURL(driver)
    # print(f'title_list: {title_list}'); print(f'url_list: {url_list}')

    # contents collecting
    mainText_list, commentCnt_list, comment_list, imgCnt_list, img_list, videoCnt_list, video_list = getContents(driver, title_list, url_list)

    # shutdown webdriver 202
    driver.quit()

    # DataFrame init
    df = pd.DataFrame({
        "title": title_list,
        "url": url_list,
        "main_text": mainText_list,
        "comment_cnt": commentCnt_list,
        "comment": comment_list,
        "img_cnt": imgCnt_list,
        "img": img_list,
        "video_cnt": videoCnt_list,
        "video": video_list,
        "ch1": ["daum" for _ in range(len(title_list))],
        "ch2": ["blog" for _ in range(len(title_list))]
    })
    print(df)

    # DataFrame 저장
    file_path = "../data/";    file_name = f"daumCafe_crawling({query}, {start_date}-{end_date}).pkl"
    df.to_pickle(file_path + file_name)

Colleting title and URL...: 100%|██████████| 10/10 [00:15<00:00,  1.51s/it]


title_list: ['기밀유지 철저한 우크라이나 군인.jpg', '단독] 정부, 국내 우크라이나 유학생 전원에 등록금·생활비 지원…7억8000만원 투입', '[펌]우크라이나 국제결혼 업체 사장이 쓴 글', '(속보)윤석열대통령, 우크라이나에 20억불 지원', '정부, 국내 우크라이나 유학생 전원에 등록금·생활비 지원…7억8000만원 투입', '현시간 우크라이나의 완벽한 GOAT.jpg', '러시아 공격 캠페인 평가, 2023년 12월 29일(우크라이나 전황)', '부활한 북한 항구, 우크라이나에서 러시아의 전쟁 지원', '우크라이나 축제 엽기사진', '우크라이나 전황 (6월공세 실패)', 'EU, 우크라이나의 가입 협상 승인... 헝가리 총리, 500억 유로 지원 거부 우크라이나 France24, euronews, Eur', '우크라이나에 봄이 오는가?/23회홍병철', '징역 5년 선고받은 우크라이나 자매', '우크라이나 전쟁: 우크라이나, ‘러시아가 하루에 우크라이나 마을 118곳 포격해’ - BBC News 코리아', '[RRN] 푸틴, 우크라이나 복제 연구소 폭격', '우크라이나 유로마이단 10주년', '교황 “전쟁의 참상... 팔레스타인, 이스라엘, 우크라이나를 잊지 맙시다”', '한국의 산에 푹 빠진 우크라이나 과학도… “그린 수소 생산에 미래 있어”', '유튜브 프리미엄 우크라이나 가족 3명 모집 👉 1명 모집중‼️', '우크라이나 재건사업 재생에너지 원전 분야 주목 기사', '본격적인 침공이 시작된 이래 최대 규모의 러시아 공습을 당한 우크라이나', '우크라이나는 크림반도에서 러시아 해군 함정을 파괴했다고 밝혔다', '미국 정보기관이 우크라이나 전쟁에 대해 거짓말을 하는 이유는 무엇입니까?', '러시아-우크라이나 전쟁 속의 북한', '우크라이나도 나몰라라 하면서 ..', '우크라이나 지방의원 의회에 수류탄 던져', '러시아-우크라이나 전쟁 상황', '몹시 곤란한 상황에 놓인 우크라이나.jpg', '전쟁이 계속 격화되는 동안 우크라이나 마을 사람

Contents...: 100%|██████████| 150/150 [04:17<00:00,  1.71s/it]


                                               title  \
0                              기밀유지 철저한 우크라이나 군인.jpg   
1    단독] 정부, 국내 우크라이나 유학생 전원에 등록금·생활비 지원…7억8000만원 투입   
2                           [펌]우크라이나 국제결혼 업체 사장이 쓴 글   
3                         (속보)윤석열대통령, 우크라이나에 20억불 지원   
4        정부, 국내 우크라이나 유학생 전원에 등록금·생활비 지원…7억8000만원 투입   
..                                               ...   
145                     [도서정보] 슬픈 우크라이나 / 김병호 / 마음친구   
146                우크라이나 재건 특수 노린다…'댐·철도' 6대 프로젝트 착수   
147                                  우크라이나의 눈물 / 사이채   
148                        룬"우크라이나에 더 큰 규모로 군수물자 지원"   
149     구 소련이자 북한에 미사일기술의 원조 우크라이나에 내년 3,600억 무상지원..   

                                                   url  \
0    https://table.cafe.daum.net/p/1080971530/25797...   
1    https://cafe.daum.net/subdued20club/ReHf/45954...   
2    https://cafe.daum.net/SoulDresser/FLTB/801099?...   
3    https://cafe.daum.net/rocksoccer/ADs1/1122715?...   
4    https://cafe.daum.net/ssaumjil/3